In [ ]:
#Code for plots

import pickle
from datetime import datetime
import numpy as np
from qutip import *
import os
from matplotlib import pyplot as plt
import sys
from scipy.integrate import solve_ivp


def load_all_dynamics(filepath):
    """
    Load multi-dynamics from saved file
    """
    with open(filepath, 'rb') as f:
        loaded_data = pickle.load(f)
    
    print(f"Multi-dynamics loaded from: {filepath}", flush=True)
    print(f"Description: {loaded_data['description']}", flush=True)
    print(f"Timestamp: {loaded_data['timestamp']}", flush=True)
    
    return (loaded_data['full_dynamics'], 
            loaded_data['redfield_dynamics'], 
            loaded_data['rates_dynamics'], 
            loaded_data['parameters'])

def plot_dynamics_from_file(filepath, t_min=None, t_max=None):
    """
    Plot dynamics from a saved file given the full file path
    with optional time range selection
    
    Parameters:
    filepath: path to the saved dynamics file
    t_min: minimum time in μs (None for start at 0)
    t_max: maximum time in μs (None for full range)
    """
    # Load the dynamics
    full_dynamics, redfield_dynamics, rates_dynamics, parameters = load_all_dynamics(filepath)
    
    # Extract parameters for plotting
    N = parameters['N']
    final_time__mus = parameters['final_time__mus']
    time_steps = parameters['time_steps']
    rabi_freq = parameters['rabi_freq']
    cavity_diss_rate = parameters['cavity_diss_rate']
    eff_coupling = parameters['eff_coupling']
    
    # Calculate expectation values
    x_hamiltonian7 = expect(tensor(qeye(N), sigmax()), full_dynamics)
    z_hamiltonian7 = expect(tensor(qeye(N), sigmaz()), full_dynamics)
    x_redfield = expect(sigmax(), redfield_dynamics)
    z_redfield = expect(sigmaz(), redfield_dynamics)
    x_rates = expect(sigmax(), rates_dynamics)
    z_rates = expect(sigmaz(), rates_dynamics)
    cavity_occupation = expect(tensor(destroy(N).dag() * destroy(N), qeye(2)), full_dynamics)

    # Create time array
    tlist = np.linspace(0, final_time__mus, time_steps)
    
    # Apply time range selection
    if t_min is None:
        t_min = 0
    if t_max is None:
        t_max = final_time__mus
    
    time_mask = (tlist >= t_min) & (tlist <= t_max)
    tlist_plot = tlist[time_mask]
    
    # Apply time mask to all data arrays
    x_hamiltonian7_plot = x_hamiltonian7[time_mask]
    z_hamiltonian7_plot = z_hamiltonian7[time_mask]
    x_redfield_plot = x_redfield[time_mask]
    z_redfield_plot = z_redfield[time_mask]
    x_rates_plot = x_rates[time_mask]
    z_rates_plot = z_rates[time_mask]
    cavity_occupation_plot = cavity_occupation[time_mask]

    fig, axes = plt.subplots(3, 1, figsize=(4, 8))
    
    # Qubit sigma_x
    axes[0].plot(tlist_plot, x_redfield_plot, color='#008000', label="Redfield")
    axes[0].plot(tlist_plot, x_hamiltonian7_plot, color='#000080', label="Hamiltonian 7")
    axes[0].plot(tlist_plot, x_rates_plot, color='#800000', label="Rates")
    axes[0].set_ylabel(r'$ <\sigma_x >$')
    axes[0].legend()
    axes[0].set_ylim(-1.1, 1.1)

    # Qubit sigma_z
    axes[1].plot(tlist_plot, z_redfield_plot, color='#008000', label="Redfield")
    axes[1].plot(tlist_plot, z_hamiltonian7_plot, color='#000080', label="Hamiltonian 7")
    axes[1].plot(tlist_plot, z_rates_plot, color='#800000', label="Rates")
    axes[1].set_ylabel(r'$<\sigma_z >$')
    axes[1].legend()
    axes[1].set_ylim(-1.1, 1.1)
    
    # Cavity occupation
    axes[2].plot(tlist_plot, cavity_occupation_plot, color='#FF8C00', label="Cavity Occupation", linewidth=2)
    axes[2].set_ylabel(r'$ \langle a^\dagger a \rangle $')
    axes[2].set_xlabel('Time (μs)')
    axes[2].legend()
    axes[2].set_ylim(bottom=0)

    params_text = []
    params_text.append(f"Rabi: {rabi_freq/(2*np.pi*1e6):.1f} MHz")
    params_text.append(f"κ: {cavity_diss_rate/(2*np.pi*1e6):.1f} MHz")
    params_text.append(f"g: {eff_coupling/(2*np.pi*1e6):.1f} MHz")
    
    if t_min != 0 or t_max != final_time__mus:
        params_text.append(f"Time: [{t_min}-{t_max}]μs")

    plt.suptitle(' | '.join(params_text), y=0.98)
    plt.tight_layout()
    plt.show()
    
    # Calculate and plot fidelities for the selected time range
    full_dynamics_plot = [full_dynamics[i] for i in range(len(full_dynamics)) if time_mask[i]]
    redfield_dynamics_plot = [redfield_dynamics[i] for i in range(len(redfield_dynamics)) if time_mask[i]]
    rates_dynamics_plot = [rates_dynamics[i] for i in range(len(rates_dynamics)) if time_mask[i]]
    
    fidelity_rates_vs_full = [fidelity(rates_dynamics_plot[i], full_dynamics_plot[i].ptrace(1)) for i in range(len(full_dynamics_plot))]
    fidelity_redfield_vs_full = [fidelity(redfield_dynamics_plot[i], full_dynamics_plot[i].ptrace(1)) for i in range(len(full_dynamics_plot))]

    # Plot the fidelities
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

    # Fidelity plot
    ax1.plot(tlist_plot, fidelity_rates_vs_full, label='Rates vs Full', linewidth=2, color='#800000')
    ax1.plot(tlist_plot, fidelity_redfield_vs_full, label='Redfield vs Full', linewidth=2, color='#008000')
    ax1.set_xlabel('Time (μs)')
    ax1.set_ylabel('Fidelity')
    ax1.legend()
    ax1.set_ylim(0, 1.05)
    ax1.set_title('Fidelity Comparison')
    ax1.grid(True, alpha=0.3)

    # Average fidelity bar plot
    methods = ['Rates vs Full', 'Redfield vs Full']
    avg_fidelities = [np.mean(fidelity_rates_vs_full), np.mean(fidelity_redfield_vs_full)]
    bars = ax2.bar(methods, avg_fidelities, color=['#800000', '#008000'])
    ax2.set_ylabel('Average Fidelity')
    ax2.set_ylim(0, 1.05)
    ax2.set_title('Average Fidelity')

    # Add value labels on bars
    for bar, value in zip(bars, avg_fidelities):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, f'{value:.3f}', 
                 ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

    # Print fidelity statistics
    print(f"Average fidelity (Rates vs Full): {np.mean(fidelity_rates_vs_full):.4f}")
    print(f"Average fidelity (Redfield vs Full): {np.mean(fidelity_redfield_vs_full):.4f}")
    print(f"Final fidelity (Rates vs Full): {fidelity_rates_vs_full[-1]:.4f}")
    print(f"Final fidelity (Redfield vs Full): {fidelity_redfield_vs_full[-1]:.4f}")

Redfield vs. Rates Presentation

